In [ ]:
# bibliotecas utilizadas
pip install lxml html5lib
pip install pandas
pip install requests

## Extract
### funcao que coleta os dados da API e transforma em DataFrame tabular.

In [1]:
import pandas as pd
import requests

# Repositorio da API: https://apidadosabertos.saude.gov.br/v1/#/Assist%C3%AAncia%20%C3%A0%20Sa%C3%BAde/get_assistencia_a_saude_registro_de_ocupacao_hospitalar_covid_19

def extract(limit=100, offset=0):
    hospitais_leitos_url = f"https://apidadosabertos.saude.gov.br/assistencia-a-saude/hospitais-e-leitos?limit={limit}&offset={offset}"
    #cabecalho para simular um navegador real
    headers = {"user-Agent": "Mozilla/5.0"}
    #Requisicao usando certificados atualizados
    response = requests.get(hospitais_leitos_url, headers=headers, verify=True)

    if response.status_code == 200:
        print("Requisicao bem-sucedida!")
        data = response.json() # retorna um dicionário com os dados
        print("Chaves do JSON:", data.keys())
        dados = data['hospitais_leitos'] # lista de registros dentro da chave
        df = pd.DataFrame(dados) # Converte em um Dataframe
        print(f"Tabela com {df.shape[0]} linhas, e {df.shape[1]} colunas")
        return df
    else:
        print(f"Erro: Codigo de status  {response.status_code}")

df = extract()

Requisicao bem-sucedida!
Chaves do JSON: dict_keys(['hospitais_leitos'])
Tabela com 100 linhas, e 31 colunas


## Transform
### funcao que ajusta colunas, tipos de dados, cria métricas 

In [2]:
def transform(df):
    
    df_leitos = df.copy() #faz uma copia do dataframe

    # elimina linhas vazias
    df_leitos.dropna(how="all", inplace=True)

    # lista de colunas do dataframe
    print(df_leitos.columns.tolist())

    # define as colunas de interesse
    df_leitos = df_leitos[[
        'nome_da_regiao_do_brasil_onde_fica_o_hospital',
        'unidade_da_federacao_onde_fica_o_hospital',
        'nome_do_municipio_onde_fica_o_hospital',
        'nome_do_hospital',
        'codigo_do_tipo_da_unidade',
        'descricao_do_tipo_da_unidade',
        'descricao_da_natureza_juridica_do_hosptial',
        'quantidade_total_de_leitos_do_hosptial',
        'quantidade_total_de_leitos_sus_do_hosptial',
    ]]

    # Renomeia colunas
    colunas_ajustadas = [ 'ds_regiao', 'sg_unidade_federacao', 'ds_municipio', 'nm_hospital', 'cd_tipo_unidade', 
                        'ds_tipo_unidade', 'ds_natureza_juridica', 'qt_leitos_hospitalares', 'qt_leitos_sus']
    df_leitos.columns = colunas_ajustadas

    # Ajusta datatypes
    df_leitos['qt_leitos_hospitalares'].astype(float)
    df_leitos['qt_leitos_sus'].astype(float)

    # Criar coluna de relação leito SUS / leito hospitalar 
    df_leitos['tx_leitos'] = ((df_leitos['qt_leitos_sus'] / df_leitos['qt_leitos_hospitalares']) * 100).apply(lambda x: f"{x:.2f}%")

    return df_leitos

df_leitos = transform(df)

['nome_da_regiao_do_brasil_onde_fica_o_hospital', 'unidade_da_federacao_onde_fica_o_hospital', 'nome_do_municipio_onde_fica_o_hospital', 'motivo_da_desabilitacao_do_hospital,_caso_esteja_desabilitado', 'nome_do_hospital', 'nome_da_razao_social_do_hospital', 'tipo_da_gestao_do_hospital', 'codigo_do_tipo_da_unidade', 'descricao_do_tipo_da_unidade', 'natureza_juridica_do_hospital', 'descricao_da_natureza_juridica_do_hosptial', 'enderco_do_hospital', 'numero_do_endereco_do_hospital', 'complemento_do_endereco_do_hospital', 'nome_do_bairro_do_endereco_do_hosptial', 'numero_do_cep_do_hospital', 'quantidade_total_de_leitos_do_hosptial', 'quantidade_total_de_leitos_sus_do_hosptial', 'quantidade_de_leitos_de_uti_do_hosptial', 'quantidade_de_leitos_de_uti_sus_do_hosptial', 'quantidade_de_leitos_de_uti_adulto_do_hosptial', 'quantidade_de_leitos_de_uti_sus_adulto_do_hosptial', 'quantidade_de_leitos_de_uti_pediatrico_do_hosptial', 'quantidade_de_leitos_de_uti_sus_pediatrico_do_hosptial', 'quantidade

In [ ]:
df_leitos[['sg_unidade_federacao', 'ds_municipio', 'nm_hospital', 'ds_tipo_unidade', 'ds_natureza_juridica', 'qt_leitos_hospitalares', 'qt_leitos_sus']]

,sg_unidade_federacao,ds_municipio,nm_hospital,ds_tipo_unidade,ds_natureza_juridica,qt_leitos_hospitalares,qt_leitos_sus
0,GO,GOIANIA,HOSPITAL DO RIM,HOSPITAL GERAL,HOSPITAL_PRIVADO,29.0,1.0
1,RS,CAIBATE,HOSPITAL ROQUE GONZALES CAIBATE,HOSPITAL GERAL,HOSPITAL_FILANTRÓPICO,41.0,31.0
2,AM,MANAUS,MATERNIDADE AZILDA DA SILVA MARREIRO,HOSPITAL ESPECIALIZADO,HOSPITAL_PÚBLICO,81.0,80.0
3,RJ,RIO DE JANEIRO,UFRJ INSTITUTO DE GINECOLOGIA,HOSPITAL GERAL,HOSPITAL_PÚBLICO,12.0,12.0
4,SP,CABREUVA,HOSPITAL SAO ROQUE CABREUVA,HOSPITAL GERAL,HOSPITAL_FILANTRÓPICO,13.0,9.0
...,...,...,...,...,...,...,...
95,PR,MEDIANEIRA,HOSPITAL E MATERNIDADE NOSSA SENHORA DA LUZ,HOSPITAL GERAL,HOSPITAL_FILANTRÓPICO,97.0,58.0
96,PI,FLORES DO PIAUI,UMS MUNICIPAL ENF MARIA DE FATIMA G FERREIRA,UNIDADE MISTA,HOSPITAL_PÚBLICO,6.0,6.0
97,DF,BRASILIA,HBDF HOSPITAL DE BASE DO DISTRITO FEDERAL,HOSPITAL GERAL,HOSPITAL_PÚBLICO,739.0,681.0
98,MT,PONTES E LACERDA,HOSPITAL VALE DO GUAPORE,HOSPITAL GERAL,HOSPITAL_PRIVADO,55.0,50.0


In [13]:
df_leitos.describe()

,qt_leitos_hospitalares,qt_leitos_sus
count,100.000000,100.000000
mean,66.310000,48.280000
std,95.221347,82.589906
min,4.000000,0.000000
25%,20.000000,10.500000
50%,34.000000,25.000000
75%,70.750000,50.250000
max,739.000000,681.000000


## Load
### Salva os dados tratados

In [109]:
def load(df_leitos):
    # Exportar para CSV 
    df_leitos.to_csv("leitos_hospitalares.csv", index=False, encoding="utf-8") 
    print("Carga concluída! Arquivo salvo como 'leitos_hospitalares.csv'")

load(df_leitos)

Carga concluída! Arquivo salvo como 'leitos_hospitalares.csv'
